## Caesar Cipher

The Caesar cipher, also known the shift cipher, is a type of substitution cipher in which each letter in the plaintext is replaced by a letter some fixed number of positions down the alphabet. 

For example, with a left shift of 3, D would be replaced by A, E would become B, and

```
Plain:    ABCDEFGHIJKLMNOPQRSTUVWXYZ
Cipher:   XYZABCDEFGHIJKLMNOPQRSTUVW
```

With the above encryption rule, a message can be encrypted as

```
Plain:  Codebuster is fun!
Cipher: Zlabyrpqbo fp crk!

```


In Codebuster competitions, a table with all possible shifts is provided. Once the shift is guessed or determined, you may use the table to quickly decrypt any given letter.

### Python implementation

Mathematically, if A=0, B=1, ... Z=25, encryption of a letter x by a shift n can be described as,

$$E_n(x) = (n+x) \mod 26 $$

while the decryption can be done by the reverse shift 

$$D_n(x) = (n-x) \mod 26 $$

In computers, lettes are ususually represented by ascii code: 'A'=65, 'B'=66, ..., 'a'=97', ... which can be obtained by the `ord()` Python function. Implementations for Caesar Encryptor/Decryptors are shown below. 

In [1]:
### Python code for Caesar Cipher

def Caesar_Encryptor(text,shift): 
    """
    Caesar Cipher to encrypt a {text} with a given {shift}
    Only letters A-Za-z are encrypted
    Return: encrypted text
    """
    # create an empty string for output
    result = "" 
  
    # iterate over the input text
    for i in range(len(text)): 
        # get the character
        char = text[i] 
        # if it's a upper case letter 
        if (char.isupper()): 
            # shift the letter (c+shift) % 26
            result += chr((ord(char) + shift- ord('A')) % 26 + ord('A')) 
        # if it's a lower case letter 
        elif (char.islower()): 
            result += chr((ord(char) + shift - ord('a')) % 26 + ord('a')) 
        # All others including space, numbers, symbols
        else:
            # just copy it
            result += char
    # return the encrypted text
    return result 

def Caesar_Decryptor(text,shift): 
    """
    Caesar Cipher to decrypt a {text} with a given {shift}
    Only letters A-Za-z are decrypted
    Return: decrypted text
    """
    # create an empty string for output
    result = "" 
  
    # iterate over the input text
    for i in range(len(text)): 
        # get the character
        char = text[i] 
        # if it's a upper case letter 
        if (char.isupper()): 
            # shift the letter (c-shift) % 26
            result += chr((ord(char) - shift- ord('A')) % 26 + ord('A')) 
        # if it's a lower case letter 
        elif (char.islower()): 
            result += chr((ord(char) - shift - ord('a')) % 26 + ord('a')) 
        # All others including space, numbers, symbols
        else:
            # just copy it
            result += char
    # return the encrypted text
    return result 

In [2]:
# Test the above functions
# Change text and/or shift for your own message,
text = "Codebuster is fun!"
shift = -3
print("Text  : " + text ) 
print("Shift : " + str(shift))
encrypted_text = Caesar_Encryptor(text,shift)
print("Cipher: " + encrypted_text)
decrypted_text = Caesar_Decryptor(encrypted_text, shift)
print("Decrypted: " + decrypted_text)

Text  : Codebuster is fun!
Shift : -3
Cipher: Zlabyrpqbo fp crk!
Decrypted: Codebuster is fun!


## Decrypt a message with two-letter words 

If there are any two-letter words in the encrypted message, it is rather easy to decrypt, since there are not so many two-letter words. 

In Caesar Cipher, while the shift is varied and unknown in decryption, the relative distance between two letters remains the same. We define the distance between two letters as the number of alphabets counted from first letter to second (not including the first). For example 'be', we count 'cde', three letters, and the distance is 3. 

In [3]:
def TwoLetterDistance(word):
    """
    """
    
    if len(word)!=2:
        print("This function can only compute the distance for two-letter words")
        return
    firstLetter = word[0].lower()
    secondLetter = word[1].lower()
    
    distance = (ord(secondLetter)-ord(firstLetter))% 26
    return distance

def TwoLetterWordDictionary():
    WordDict = {}
    TwoLetterWordList="of, to, in, it, is, be, as, at, so, we, he, by, or, on, do, if, me, my, up, an, go, no, us, am".split(', ')
    for word in TwoLetterWordList:
        distance = TwoLetterDistance(word)
        WordDict.update({word : distance})
    WordDict = dict(sorted(WordDict.items(), key=lambda item: item[1]))
    return WordDict    
        
MyTwoLetterWordDictionary = TwoLetterWordDictionary()
for word,distance in MyTwoLetterWordDictionary.items():
    print("Word ", word, "has a distance=", distance, ", reverse distance=", distance-26)

Word  no has a distance= 1 , reverse distance= -25
Word  be has a distance= 3 , reverse distance= -23
Word  or has a distance= 3 , reverse distance= -23
Word  in has a distance= 5 , reverse distance= -21
Word  we has a distance= 8 , reverse distance= -18
Word  go has a distance= 8 , reverse distance= -18
Word  is has a distance= 10 , reverse distance= -16
Word  it has a distance= 11 , reverse distance= -15
Word  do has a distance= 11 , reverse distance= -15
Word  my has a distance= 12 , reverse distance= -14
Word  am has a distance= 12 , reverse distance= -14
Word  an has a distance= 13 , reverse distance= -13
Word  of has a distance= 17 , reverse distance= -9
Word  as has a distance= 18 , reverse distance= -8
Word  me has a distance= 18 , reverse distance= -8
Word  at has a distance= 19 , reverse distance= -7
Word  to has a distance= 21 , reverse distance= -5
Word  up has a distance= 21 , reverse distance= -5
Word  so has a distance= 22 , reverse distance= -4
Word  he has a distance= 

For example, in the encrypted message "Zlabyrpqbo fp crk!", there is a two-letter word "fp". The distance between 'f' and 'p' is 10. We can easily figure out that the word must be "is". The shift is then the distance between 'i' and 'f', which is 23, or -3. We now can solve the puzzle, with the python code below

In [4]:
# find the distance between two letters in the two-letter word
twoLetterWord = 'fp'
distance=TwoLetterDistance(twoLetterWord) # return 10
print("Distance between ", twoLetterWord, " is ", distance)
# Check the two-letter word dictionary and possible matches
for word,dist in MyTwoLetterWordDictionary.items(): 
    if dist == distance:
        print("A possible match is '"+word+"' with the shift ", TwoLetterDistance(word[0]+twoLetterWord[0]))    
# return 'is' with the shift 23
# we now use the shift to decrypt
shift=23
decrypted_text = Caesar_Decryptor("Zlabyrpqbo fp crk!", shift)
print("Decypted: ", decrypted_text)

Distance between  fp  is  10
A possible match is 'is' with the shift  23
Decypted:  Codebuster is fun!


In [5]:
def Caesar_Decryptor_TwoLetterWord(encrypted_text, twoLetterWord):
    """
    Caesar Decryptor to use a two-letter word to decrypt a message
    """
    
    # compute the distance between the two letters in twoLetterWord
    distance=TwoLetterDistance(twoLetterWord)
    print("Distance between ", twoLetterWord, " is ", distance)
    
    # create a list of all possible decrypted messages
    result = []
    
    # Check the two-letter word dictionary and possible matches
    for word,dist in MyTwoLetterWordDictionary.items(): 
        # found a matched two-letter word in dictionary
        if dist == distance:
            # calculate the shift
            shift = TwoLetterDistance(word[0]+twoLetterWord[0])
            print("A possible match is '"+word+"' with the shift ", shift)
            # decrypt the message
            decrypted_text = Caesar_Decryptor(encrypted_text, shift)
            print("The decrypted message is "+decrypted_text)
            # append the decrypted text to the answer list
            result.append(decrypted_text)
    # return the list of all possible answers
    return result
    
# test 
print(Caesar_Decryptor_TwoLetterWord("Zlabyrpqbo fp crk!", "fp"))

# more tests with two or more possibilities
print(Caesar_Decryptor_TwoLetterWord('SQJSX CU YV OEK SQD', 'CU'))
print(Caesar_Decryptor_TwoLetterWord('SQJSX CU YV OEK SQD', 'YV'))

Distance between  fp  is  10
A possible match is 'is' with the shift  23
The decrypted message is Codebuster is fun!
['Codebuster is fun!']
Distance between  CU  is  18
A possible match is 'as' with the shift  2
The decrypted message is QOHQV AS WT MCI QOB
A possible match is 'me' with the shift  16
The decrypted message is CATCH ME IF YOU CAN
['QOHQV AS WT MCI QOB', 'CATCH ME IF YOU CAN']
Distance between  YV  is  23
A possible match is 'he' with the shift  17
The decrypted message is BZSBG LD HE XNT BZM
A possible match is 'by' with the shift  23
The decrypted message is VTMVA FX BY RHN VTG
A possible match is 'if' with the shift  16
The decrypted message is CATCH ME IF YOU CAN
['BZSBG LD HE XNT BZM', 'VTMVA FX BY RHN VTG', 'CATCH ME IF YOU CAN']


In [6]:
# make your own test here
import random
shift = random.randint(-26,26)
encrypted = Caesar_Encryptor("My test here", shift)
twoLetterWord_encrypted = encrypted.split(' ')[0]
decrypted = Caesar_Decryptor_TwoLetterWord(encrypted, twoLetterWord_encrypted)
print("Cipher: ", encrypted)
print("Decrypted: ", decrypted)

Distance between  Co  is  12
A possible match is 'my' with the shift  16
The decrypted message is My test here
A possible match is 'am' with the shift  2
The decrypted message is Am hsgh vsfs
Cipher:  Co juij xuhu
Decrypted:  ['My test here', 'Am hsgh vsfs']


### Decipher a message with other patterns

In Cryptanalysis, some of the strategies are 

* Identify Common Pairs Of Letters
* Identify The Smallest Words First
* Tailor Made Frequency Tables
* Play The Guessing Game

More details as well as the frequent patterns in English can be found [here](https://www3.nd.edu/~busiforc/handouts/cryptography/cryptography%20hints.html).


### Practice Examples

1. 'Bpm ozmibmab otwzg qv tqdqvo tqma vwb qv vmdmz nittqvo, jcb qv zqaqvo mdmzg bqum em nitt.'

2. 'Vjg yca vq igv uvctvgf ku vq swkv vcnmkpi cpf dgikp fqkpi.'

3. 'Dolu fvb ylhjo aol luk vm fvby yvwl, apl h ruva pu pa huk ohun vu.'

4. 'Itaqhqd ue tmbbk iuxx ymwq aftqde tmbbk faa.'

5. 'Vgrvtn mzhzhwzm ocvo tjp vmz vwnjgpozgt pidlpz. Epno gdfz zqzmtjiz zgnz.'


see the answers [here](Answer.txt).